In [ ]:
import instructor
import llama_cpp

# from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
from pydantic import BaseModel, Field

llama = llama_cpp.Llama(
    model_path="../models/Hermes-2-Pro-Llama-3-8B-Q4.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    chat_format="llama-3",
    verbose=False,
)


create = instructor.patch(
    create=llama.create_chat_completion_openai_v1,
    mode=instructor.Mode.JSON_SCHEMA,
)

In [ ]:
class QuestionNaming(BaseModel):
    chain_of_thought: str = Field(
        ...,
        description="The chain of thought that led to the prediction.",
    )
    name: str = Field(
        ...,
        description="Code that the user will implement, such as blank function implementations for the user to implement. Use the question prompt to help determine what the user needs to implement.",
    )


def generate_question_name(data: str) -> QuestionNaming:
    extraction: QuestionNaming = create(
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert at extracting information from exams. "
                    "You will be given a question from a Computer Science exam, "
                    "and you will need to generate a name for the question. "
                    "Follow a leetcode naming convention. "
                    "Do not solve the question. "
                ),
            },
            {
                "role": "user",
                "content": (
                    f"Extract name from the following text: " f"<text>{data}</text>"
                ),
            },
        ],
        response_model=QuestionNaming,
    )
    return extraction

In [ ]:
from parser.model import Section
from parser.parse import main
from typing import List

sections: List[Section] = main("../fe_files/exams/FE-Aug23.pdf")


for section in sections:
    for question in section.questions:
        input_question = question.text
        print(input_question)
        generated_name: str = generate_question_name(input_question)
        question.generated_name = generated_name


class Document(BaseModel):
    sections: List[Section]


document = Document(sections=sections)

# Write pydantic models to JSON file
with open("document_named.json", "w") as json_file:
    json_file.write(document.model_dump_json())